In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
import sys

sys.path.insert(0, '../..')

## Load Dataset

In [4]:
from entity_embed.benchmarks import ITunesAmazonStructuredBenchmark

benchmark = ITunesAmazonStructuredBenchmark(data_dir_path="../data/")
benchmark

22:15:08 INFO:Extracting iTunes-Amazon-Structured...
22:15:08 INFO:Reading iTunes-Amazon-Structured record_dict...
22:15:09 INFO:Reading iTunes-Amazon-Structured train.csv...
22:15:09 INFO:Reading iTunes-Amazon-Structured valid.csv...
22:15:09 INFO:Reading iTunes-Amazon-Structured test.csv...


<ITunesAmazonStructuredBenchmark> from http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured/iTunes-Amazon/itunes_amazon_exp_data.zip

## Preprocess

In [5]:
field_list = ['Song_Name', 'Artist_Name', 'Album_Name', 'Genre', 'Price', 'CopyRight', 'Time', 'Released']

In [6]:
import unidecode

def clean_str(s):
    return unidecode.unidecode(s).lower().strip()

for record_dict in [benchmark.train_record_dict, benchmark.valid_record_dict, benchmark.test_record_dict]:
    for record in record_dict.values():
        for field in field_list:
            record[field] = clean_str(record[field])

## Init Data Module

In [7]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [8]:
from entity_embed import PairNumericalizer

pair_numericalizer = PairNumericalizer(field_list)

In [9]:
batch_size = 32
eval_batch_size = 256
datamodule = benchmark.build_matcher_datamodule(
    pair_numericalizer=pair_numericalizer,
    batch_size=batch_size,
    eval_batch_size=eval_batch_size,
    random_seed=random_seed
)

## Training

In [10]:
from entity_embed import Matcher

model = Matcher(
    pair_numericalizer=pair_numericalizer
)

22:15:12 INFO:Load pretrained SentenceTransformer: stsb-distilbert-base
22:15:12 INFO:Did not find folder stsb-distilbert-base
22:15:12 INFO:Search model on server: http://sbert.net/models/stsb-distilbert-base.zip
22:15:12 INFO:Load SentenceTransformer from folder: /home/fjsj/.cache/torch/sentence_transformers/sbert.net_models_stsb-distilbert-base
22:15:13 INFO:Use pytorch device: cuda


In [11]:
trainer = model.fit(
    datamodule,
    min_epochs=5,
    max_epochs=100,
    check_val_every_n_epoch=1,
    early_stop_monitor="valid_f1_at_0.5",
    tb_save_dir='../tb_logs',
    tb_name=f'matcher-{benchmark.dataset_name}'
)

22:15:13 INFO:GPU available: True, used: True
22:15:13 INFO:TPU available: False, using: 0 TPU cores
22:15:13 INFO:Train positive pair count: 78
22:15:13 INFO:Train negative pair count: 243
22:15:13 INFO:Valid positive pair count: 27
22:15:13 INFO:Valid positive pair count: 82
22:15:13 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
22:15:16 INFO:
  | Name        | Type              | Params
--------------------------------------------------
0 | matcher_net | MatcherNet        | 66.4 M
1 | loss_fn     | BCEWithLogitsLoss | 0     
--------------------------------------------------
66.4 M    Trainable params
0         Non-trainable params
66.4 M    Total params
265.455   Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

22:19:13 INFO:Loading the best validation model from ../tb_logs/matcher-iTunes-Amazon-Structured/version_0/checkpoints/epoch=12-step=142.ckpt...
22:19:13 INFO:Load pretrained SentenceTransformer: stsb-distilbert-base
22:19:13 INFO:Did not find folder stsb-distilbert-base
22:19:13 INFO:Search model on server: http://sbert.net/models/stsb-distilbert-base.zip
22:19:13 INFO:Load SentenceTransformer from folder: /home/fjsj/.cache/torch/sentence_transformers/sbert.net_models_stsb-distilbert-base
22:19:14 INFO:Use pytorch device: cuda


In [12]:
model.validate(datamodule)

{'valid_f1_at_0.3': tensor(0.8679),
 'valid_f1_at_0.5': tensor(0.8846),
 'valid_f1_at_0.7': tensor(0.8400),
 'valid_f1_at_0.9': tensor(0.6341),
 'valid_precision_at_0.3': tensor(0.8846),
 'valid_precision_at_0.5': tensor(0.9200),
 'valid_precision_at_0.7': tensor(0.9130),
 'valid_precision_at_0.9': tensor(0.9286),
 'valid_recall_at_0.3': tensor(0.8519),
 'valid_recall_at_0.5': tensor(0.8519),
 'valid_recall_at_0.7': tensor(0.7778),
 'valid_recall_at_0.9': tensor(0.4815)}

## Testing

In [13]:
model.test(datamodule)

22:19:21 INFO:Test positive pair count: 27
22:19:21 INFO:Test positive pair count: 82


{'test_f1_at_0.3': tensor(0.9091),
 'test_f1_at_0.5': tensor(0.8889),
 'test_f1_at_0.7': tensor(0.8846),
 'test_f1_at_0.9': tensor(0.8333),
 'test_precision_at_0.3': tensor(0.8929),
 'test_precision_at_0.5': tensor(0.8889),
 'test_precision_at_0.7': tensor(0.9200),
 'test_precision_at_0.9': tensor(0.9524),
 'test_recall_at_0.3': tensor(0.9259),
 'test_recall_at_0.5': tensor(0.8889),
 'test_recall_at_0.7': tensor(0.8519),
 'test_recall_at_0.9': tensor(0.7407)}